In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from mlxtend.data import loadlocal_mnist
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
x_train = pd.read_csv('C:/Users/Muhammad Darab/OneDrive/Desktop/Online Course Files/Machine Learning/Data Sets/DecisionTree/DecisionTree.csv', usecols = ['Sales', 'CompPrice', 'Income', 'Advertising', 'Population', 'Price', 'ShelveLoc', 'Age', 'Education'])
y_train = pd.read_csv('C:/Users/Muhammad Darab/OneDrive/Desktop/Online Course Files/Machine Learning/Data Sets/DecisionTree/DecisionTree.csv', usecols = ['Urban'])
x_train_1 = pd.read_csv('C:/Users/Muhammad Darab/OneDrive/Desktop/Online Course Files/Machine Learning/Data Sets/DecisionTree/DecisionTree-1.csv', usecols = ['Brown Cap', 'Stalk Shape', 'Solitary'])
y_train_1 = pd.read_csv('C:/Users/Muhammad Darab/OneDrive/Desktop/Online Course Files/Machine Learning/Data Sets/DecisionTree/DecisionTree-1.csv', usecols = ['Edible'])
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_train_1 = x_train_1.to_numpy()
y_train_1 = y_train_1.to_numpy()
m, n = x_train.shape
for i in range(m):
    for j in range(n):
        if x_train[i][j] == True:
            x_train[i][j] = 1
        elif x_train[i][j] == False:
            x_train[i][j] = 0
x_train = x_train[:181]
y_train = y_train[:181]
x_train = x_train[:130]
y_train = y_train[:130]

[[1 1 1 ... 0 1 0]
 [1 0 0 ... 1 0 1]
 [0 1 0 ... 0 0 0]
 ...
 [1 0 0 ... 1 1 1]
 [1 0 1 ... 1 0 1]
 [1 1 1 ... 1 0 1]]


In [143]:
def entropy(Y):
    m = len(Y)
    if m == 0:
        return 0
    p_1 = 0
    for i in range(m):
        if Y[i] == 1:
            p_1 += 1
    p_1 = p_1 / m
    if p_1 == 0 or p_1 == 1:
        return 0
    entropy = -(p_1) * np.log2(p_1) - (1 - p_1) * np.log2(1 - p_1)
    return entropy

In [144]:
def splitDataSet(X, node_indices, feature):
    left_indices = []
    right_indices = []
    for i in node_indices:
        if X[i][feature] == 1:
            left_indices.append(i)
        elif X[i][feature] == 0:
            right_indices.append(i)
    return left_indices, right_indices

In [145]:
node_indices = []
node_indices_1 = []
for i in range(len(x_train)):
    node_indices.append(i)
for i in range(len(x_train_1)):
    node_indices_1.append(i)

In [146]:
def informationGain(X, Y, node_indices, feature):
    left_indices, right_indices = splitDataSet(X, node_indices, feature)
    X_node, y_node = X[node_indices], Y[node_indices]
    X_left, y_left = X[left_indices], Y[left_indices]
    X_right, y_right = X[right_indices], Y[right_indices]
    if len(X_left) == 0 or len(X_right) == 0:
        return 0
    p_1_l = y_left
    p_1_r = y_right
    w_l = len(X_left) / len(X_node)
    w_r = len(X_right) / len(X_node)
    information_gain = entropy(Y) - (w_l * entropy(p_1_l) + w_r * entropy(p_1_r))
    return information_gain

In [147]:
def getBestSplit(X, Y, node_indices):
    m, n = X.shape
    bft = -1
    for i in range(n):
        bft1 = informationGain(X, Y, node_indices, i)
        if bft < bft1:
            best_feature = i
            bft = bft1
    return best_feature

In [148]:
tree = []
def buildTree(X, Y, node_indices, branch_name, max_depth, current_depth):
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
    best_feature = getBestSplit(X, Y, node_indices) 
    tree.append((current_depth, branch_name, best_feature, node_indices))
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    left_indices, right_indices = splitDataSet(X, node_indices, best_feature)
    buildTree(X, Y, left_indices, "Left", max_depth, current_depth + 1)
    buildTree(X, Y, right_indices, "Right", max_depth, current_depth + 1)

In [150]:
buildTree(x_train_1, y_train_1, node_indices_1, 'Root', 2, 0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]


In [156]:
buildTree(x_train, y_train, node_indices, 'Root', 3, 0)

 Depth 0, Root: Split on feature: 6
- Depth 1, Left: Split on feature: 2
-- Depth 2, Left: Split on feature: 0
   --- Left leaf node with indices [3, 4, 17, 21, 37, 41, 47, 58, 62, 70, 89, 94, 99, 107, 116, 123, 128, 129]
   --- Right leaf node with indices [16, 23, 26, 30, 32, 33, 42, 52, 53, 54, 60, 63, 80, 85, 93, 102, 108, 110]
-- Depth 2, Right: Split on feature: 2
   --- Left leaf node with indices []
   --- Right leaf node with indices [1, 7, 9, 12, 14, 19, 24, 27, 29, 35, 36, 44, 46, 48, 51, 59, 71, 78, 82, 83, 84, 86, 91, 96, 100, 101, 103, 104, 106, 111, 112, 124, 127]
- Depth 1, Right: Split on feature: 6
-- Depth 2, Left: Split on feature: 0
   --- Left leaf node with indices []
   --- Right leaf node with indices []
-- Depth 2, Right: Split on feature: 6
   --- Left leaf node with indices []
   --- Right leaf node with indices [0, 2, 5, 6, 8, 10, 11, 13, 15, 18, 20, 22, 25, 28, 31, 34, 38, 39, 40, 43, 45, 49, 50, 55, 56, 57, 61, 64, 65, 66, 67, 68, 69, 72, 73, 74, 75, 76, 